In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from rag_consts import *
from vector_db import *

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

# Overview

Idea:
* input:
    *  "interview_description": "opis wizyty...",
    * "treatment_description": "badanie...",
* output:
    * "applied_medicines"
    * "recommendation"

# 1. Get Visits

In [ ]:
# TODO fix: use clients code

import requests
# from sql_db.visit import Visit

SERVER_URL = "http://localhost:8000"


def get_visits():
    r = requests.get(f"{SERVER_URL}/visits")
    answ = r.json()
    return answ

visits = get_visits()
print(f"visits: {visits}")

In [ ]:
with open(VISITS_FILENAME, 'w', encoding='utf-8') as f:
    json.dump(visits, f, ensure_ascii=False, indent=2)

# 2. Build index

In [ ]:
embedding_list, id_to_visit = process_visits(client, visits)

In [ ]:
index = build_index(embedding_list)

In [ ]:
faiss.write_index(index, INDEX_FILENAME)
print(f"Saved in '{INDEX_FILENAME}'")

In [ ]:
with open(ID_TO_VISIT_FILENAME, "wb") as f:
    pickle.dump(id_to_visit, f)

# 3. Use Index

In [ ]:
with open(VISITS_FILENAME, 'r', encoding='utf-8') as f:
    visits = json.load(f)

loaded_index = faiss.read_index(INDEX_FILENAME)
with open(ID_TO_VISIT_FILENAME, "rb") as f:
    loaded_id_to_visit = pickle.load(f)

# 4. Test

In [ ]:
test_dict = {
    "interview_description": "Dziś pierwszy raz napady drgawkowe z niecałkowitą utratą przytomności, pierwszy o godzinie 18:20. W trakcie ataku pies otwierał szeroko oczy i głęboko oddychał, pomiędzy atakami całkowita utrata przytomności, nie oddał ani kału ani moczu. Każdy z ataków trwał około 30-40 sekund. Do godziny 21:00 około 7 ataków. Ostatni posiłek jadł rano, ma stały dostęp do karmy. Pies nie ma dostępu do trucizn ani nawozów, mieszka w kojcu, dziś rano jadł standardowe jedzenie. Około miesiąc temu przebył babeszjozę potwierdzoną w gabinecie weterynaryjnym, otrzymał dwie kroplówki dożylne, nie kontynuowano leczenia. Wcześniej nie wykazywał niepokojących objawów. Jest psem adoptowanym, właściciele nie znają jego wcześniejszych losów, w schronisku nie miał ataków, wcześniej silne zapalenie płuc. Zaszczepiony przeciwko wściekliźnie i chorobom wirusowym. Pies jest agresywny. Dziś przywieziony nieprzytomny, mokry, właścicielka schładzała polewając wodą.",
    "treatment_description": "Podłączono psa do maski tlenowej, założono wkłucie dożylne, pobrano krew na szeroki profil z jonogramem, podano Płyn Ringera 600 ml w tempie 400 ml/h, w trakcie ataku podano propofol 4,5 ml i.v., następnie dodatkowo 1 ml propofolu i.v. Odbyły się badania laboratoryjne (krew, T4, fT4), monitorowano parametry życiowe. Przygotowano psa do wyjazdu do lecznicy całodobowej na monitoring.",
    "applied_medicines": "Propofol-Lipuro 10mg/1ml 5,5 ml, Płyn Ringera 600 ml",
    "recommendation": "Nocny monitoring w lecznicy całodobowej, diagnostyka w celu wykluczenia zaburzeń kardiologicznych oraz pełne badanie neurologiczne po odzyskaniu przez psa przytomności. Oczekiwanie na wynik oznaczenia hormonów tarczycy. Kontakt w tej sprawie z lecznicą."
}

In [ ]:
interview_description = test_dict["interview_description"]
treatment_description = test_dict["treatment_description"]
k = 3

In [ ]:
distances, indices = find_in_vector_db(interview_description, treatment_description, k, client, loaded_index)
distances, indices

In [ ]:
similar_visits_info = get_similar_visits_info(indices, visits, loaded_id_to_visit)
similar_visits_info

In [ ]:
prompt = generate_prompt_for_new_visit(interview_description, treatment_description, k, visits, client, loaded_index, loaded_id_to_visit)
print(prompt)

In [ ]:
rec_data = get_recommendations(interview_description, treatment_description, api_key=OPENAI_API_KEY)

In [ ]:
print(rec_data)